# TP 5 - Visión por Computadora 1

## Enunciado:

### - Objetivo
- Implementar el detector de fondo naive usando la mediana como estimador El algoritmo debe recibir el parámetro N (cantidad de frames utilizados para la estimación) y el intervalo de tiempo para recalcular el fondo
- Se deben generar las mascaras de foreground y aplicarlas a los frames para segmentar los objetos en movimiento
- Comparar con alguno de los métodos vistos en la practica basados en mezcla de gaussianas

***

### Funciones y variables globales

In [1]:
%matplotlib inline

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

import random
import time

In [2]:
n_frames = 20
n_time = 10
filename = 'vtest.avi'

### Funciones auxiliares

In [3]:
def get_random_indexes(n_items, actual_index, max_frames):  
    index = actual_index + n_items
    index = min(index, max_frames)    
    indexes = np.random.choice(index, size=n_items, replace=False)
    return indexes

In [4]:
def get_background(n_frames, index, total_frames):
    frames = []
    indexes = get_random_indexes(n_frames, index, total_frames)
    for i in indexes:
        capture.set(1, i)
        ret, frame = capture.read()
        frames.append(frame)

    return np.median(frames, axis=0).astype(dtype=np.uint8)

### Implementación algoritmo Naive

In [5]:
def naive_background_substractor(n_frames, n_time, capture, apply_opening=False):
    total_frames = int(capture.get(7))
    background = get_background(n_frames, 0, total_frames)
    while True:
        # Obtenemos el número del frame actual
        index = capture.get(cv.CAP_PROP_POS_FRAMES)

        # Recalculamos el background luego de n número de frames
        if index % n_time == 0:
            background = get_background(n_frames, 0, total_frames)
            
        # Leemos un frame
        index += 1
        index = min(index, total_frames)
        capture.set(1, index)   
        ret, frame = capture.read()
        if frame is None:
            break
        
        # Restamos el frame actual con la mediana y binarizamos para obtener el foregroumd
        diff_frame = cv.absdiff(frame, background)
        _, foreground = cv.threshold(diff_frame, 25, 255, cv.THRESH_BINARY)
        
        if apply_opening:
            kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))
            foreground = cv.morphologyEx(foreground, cv.MORPH_OPEN, kernel)

        # Escribimos sobre la imagen el número de frame procesado
        cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
        cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
                   cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
        
        
        # mostramos frame original e imagen binaria background/foreground
        cv.imshow('Frame', frame)
        cv.imshow('FG Mask', foreground)

        # Corremos hasta que termine o apriete escape
        keyboard = cv.waitKey(30)
        if keyboard == 'q' or keyboard == 27:
            break

    cv.destroyAllWindows()
    capture.release()

### Prueba del algoritmo

In [6]:
capture = cv.VideoCapture(filename)

start = time.time()
naive_background_substractor(n_frames, n_time, capture)
end = time.time()

processed_time = end - start
print(f"Tiempo de procesamiento: {processed_time} segundos.")

Tiempo de procesamiento: 50.935993671417236 segundos.


### Comparación con algoritmo de mezcla de gausianas

In [7]:
def mog2_background_substractor(capture):
    # Definimos el método a utilizar
    backSub = cv.createBackgroundSubtractorMOG2()

    while True:
        # Leemos un frame
        ret, frame = capture.read()
        if frame is None:
            break

        # Aplicamos la sustracción al frame leído
        fgMask = backSub.apply(frame)

        # Escribimos sobre la imagen el número de frame procesado
        cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
        cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
                   cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))

        # mostramos frame original e imagen binaria background/foreground
        cv.imshow('Frame', frame)
        cv.imshow('FG Mask', fgMask)

        # Corresmos hasta que termine o apriete escape
        keyboard = cv.waitKey(30)
        if keyboard == 'q' or keyboard == 27:
            break

    cv.destroyAllWindows()
    capture.release()

In [8]:
capture = cv.VideoCapture(filename)

start = time.time()
mog2_background_substractor(capture)
end = time.time()

processed_time = end - start
print(f"Tiempo de procesamiento: {processed_time} segundos.")

Tiempo de procesamiento: 60.95148515701294 segundos.


### Conclusiones

Los resultados obtenidos empleando el extractor de fondos naive, resultan sorprendentes dada la simplicidad del algoritmo. 
El mismo fue capaz de extraer el fondo del video de pruebas sin inconvenientes. Como posibles mejoras futuras se propone emplear frames aleatorios cercanos al frame actual para computar el background.

Por otro lado, al comparar el algoritmo contra el de MOG2, podemos observar que el extractor de fondos naive se desempeña de manera más veloz, necesitando 50.94 segundos para procesar el video, mientras que el algoritmo MOG2 necesitó 60.95 segundos.
No obstante, si empleamos procesamiento morfológico (operacón de apertura), es posible notar como se degrada la performance del algoritmo en cuanto a tiempo, demorando  97.79 segundos, sin embargo con esta funcionalidad activada, la extracción de fondos mejora.